# Robin Hood Trader version 1.0

In [1]:
#Install the Robinhood  API Framework robin-stocks
!pip install click
!pip install twilio
!pip install robin-stocks
!python -m pip install --upgrade pip


Requirement already up-to-date: pip in c:\users\alarc\anaconda3\lib\site-packages (20.0.2)


### Set up Robin Hood Account

In [ ]:
import robin_stocks as r
import  pandas as pd
from datetime import datetime
from twilio.rest import Client

#Set up twilio for text messaging gains and buys
client = Client("AC80b6d17d9b24b3333d3a43dad77cd6f6", "dea1e6fae4426a1775cb609c39008559")
#Set up robin-stocks api login info
login = r.login('alarconfer12@gmail.com','Mirey12345')

ERROR: There was an issue loading pickle file. Authentication may be expired - logging in normally.


## Checking out my stocks

In [ ]:
my_stocks = pd.DataFrame(r.build_holdings())
my_stocks

## Checking for the Fear and Greed Score on CNN

In [ ]:
#Installing and importing the necessary modules.
!pip install requests
!pip install bs4
import requests 
from bs4 import BeautifulSoup 

In [ ]:
FEAR_AND_GREED_URL = 'https://money.cnn.com/data/fear-and-greed/'
F_N_G_PRHASE = 'Fear &amp; Greed Now: '
request = requests.get(FEAR_AND_GREED_URL)
html = request.content
soup = BeautifulSoup(html)
div = soup.find_all(id='needleChart')
div = str(div[0])
position = div.find(F_N_G_PRHASE)
position = position + len(F_N_G_PRHASE)
current_fear_greed_score = int(div[position:position + 3].strip())
print(current_fear_greed_score)

### Robin Hood App 

In [ ]:
my_crypto_currencies = r.crypto.get_crypto_positions()
pd.DataFrame(my_crypto_currencies)

In [ ]:
crypto_names = [val['currency']['code'] for val in my_crypto_currencies]
crypto_names

In [ ]:
account = r.profiles.load_account_profile()
account

In [ ]:
print(float(account['buying_power']) / float(r.crypto.get_crypto_quote("DOGE")['ask_price']))
def payout():
    r.orders.order_sell_crypto_by_quantity(crypto_names[name], my_crypto_currencies[name]['quantity'], priceType='mark_price')

In [ ]:
#r.orders.order_buy_crypto_by_price('LTC', float(account['buying_power']) * 0.99, priceType='mark_price', timeInForce='gtc')

### Cryptocurrency Trader 2.5:

In [ ]:
###### Create a list with all of the cryptocurrency codes necessary to access their cryptocurrency information on the robin hood API
crypto_names = [val['currency']['code'] for val in my_crypto_currencies]

#Import the time library to be able to run the algorithm for a certain set of time.
import time

#Variables for common time data_types
hour = (60 * 60)
day = hour * 24
month = day * 30

#Create a dictionary with the key being the cryptocurrency code and the value being the initial buying value 
buying_prices = {'initial_DOGE_price': 0.00274,
                 'initial_ETC_price': 10.37,
                  'initial_BTC_price': 8750,
                  'initial_LTC_price': 53.26,
                  'initial_BSV_price': 390,
                  'initial_ETH_price': 170,
                  'initial_BCH_price': 380
                }

#Create a dicitionary with a value corresponding to the qunatity of coins of each crypto
quantities = {'DOGE_quantity': my_crypto_currencies[6]['quantity'],
              'ETC_quantity': my_crypto_currencies[4]['quantity'],
              'BTC_quantity': my_crypto_currencies[3]['quantity'],
              'BCH_quantity': my_crypto_currencies[0]['quantity'],
              'LTC_quantity': my_crypto_currencies[1]['quantity'],
              'BSV_quantity': my_crypto_currencies[5]['quantity'],
              'ETH_quantity': my_crypto_currencies[2]['quantity']
             }

#Create a end point
end = time.time() + month #Using a month at a time due to this being a test

#Create a dictionary to keep the buying thresholds for each stock
crypto_buying_thresholds = {}


while time.time() < end:
    try:
        try:
            #Create a dictionary to keep track of the asking price for each stock
            my_cryptos_asking_price = {}
            #Loop over all the names inside of the crypto_names list
            for name in range(len(crypto_names)):
                #Keep track of old price
                old_price = buying_prices['initial_' + crypto_names[name] + '_price']
                #Get the price for the crypto in question and add it to the brypto_asking_price dictionary
                ask_crypto_price = r.crypto.get_crypto_quote(crypto_names[name])['mark_price']
                print(crypto_names[name] + ': ' + str(ask_crypto_price) + '\n' + 'sold: ' , float(r.crypto.get_crypto_quote(crypto_names[name])['mark_price']) >= (buying_prices['initial_' + crypto_names[name] + '_price'] * 1.025))
                my_cryptos_asking_price[crypto_names[name]] = ask_crypto_price
                sell_crypto = {'sell': float(ask_crypto_price) >= (buying_prices['initial_' + crypto_names[name] + '_price'] * 1.2)}
                if float(my_crypto_currencies[name]['quantity']) > 0:
                    #if price gets below a certain price point sell as to not be stuck holding bag for a substantial amount of time.
                    if (float(r.crypto.get_crypto_quote(crypto_names[name])['mark_price']) <= (buying_prices['initial_' + crypto_names[name] + '_price'] * .85)):
                        #Sell my crypto coins
                        r.orders.order_sell_crypto_by_quantity(crypto_names[name],float(my_crypto_currencies[name]['quantity']), priceType='bid_price')
                    #If the current asking price is more than or equal to the selling threshold
                    if (float(r.crypto.get_crypto_quote(crypto_names[name])['mark_price']) >= (buying_prices['initial_' + crypto_names[name] + '_price'] * 1.25)):
                        print('sale of ' + crypto_names[name] + ' made at a gain!')
                        #Get the current crypto price
                        current_crypto_price = float(ask_crypto_price)
                        sale_price = {'sale_price': current_crypto_price}
                        #Sell my crypto coins
                        r.orders.order_sell_crypto_by_quantity(crypto_names[name],float(my_crypto_currencies[name]['quantity']), priceType='bid_price')
                        print(((float(ask_crypto_price) >= (buying_prices['initial_' + crypto_names[name] + '_price'] * 1.0014)) or (float(ask_crypto_price) < (buying_prices['initial_' + crypto_names[name] + '_price'] * .9))) and float(my_crypto_currencies[name]['quantity']) > 0.00
    )
                        #Set a buy threshold for crypto coin
                        buy_threshold = (old_price + current_crypto_price) / 2
                        #Store buying threhshold in the crypto_buying_threshold dictionary
                        crypto_buying_thresholds[crypto_names[name]] = buy_threshold
                        #print('Made a sell of ' + crypto_names[name] + 'worth: ' + str(current_crypto_price - buying_prices['initial_' + crypto_names[name] + '_price']))
                #If thre are any elements inside of the crypto_buying_thresholds dictionary and the name of the crypto is inside of the dictionary
                if (len(crypto_buying_thresholds) > 0) and (crypto_names[name] in crypto_buying_thresholds):
                #If the current_crypto_price is less than the threshold set for buying crypto
                    print('Made buy for ' + crypto_names[name] + ' = ' + str((current_crypto_price <= crypto_buying_thresholds[crypto_names[name]]) and (float(account['buying_power'])> 0.06)))
                    if (current_crypto_price <= crypto_buying_thresholds[crypto_names[name]]) and (float(account['buying_power'])> 0.06):
                        #Get the current price of crypto price
                        buying_prices['initial_' + crypto_names[name] + '_price'] = float(ask_crypto_price)
                        #Buy the named crypto
                        r.orders.order_buy_crypto_by_price(crypto_names[name], float(account['buying_power']) * 0.99, priceType='mark_price', timeInForce='gtc')
                print('time: ' + str(datetime.now())) 
        except TypeError:
            pass
    except ValueError:
        pass

### Cryptocurrency Trader 2.0:

In [ ]:
#### Create a list with all of the cryptocurrency codes necessary to access their cryptocurrency information on the robin hood API
crypto_names = [val['currency']['code'] for val in my_crypto_currencies]

#Import the time library to be able to run the algorithm for a certain set of time.
import time

#Variables for common time data_types
hour = (60 * 60)
day = hour * 24
month = day * 30

#Create a dictionary with the key being the cryptocurrency code and the value being the initial buying value 
buying_prices = {'initial_DOGE_price': 0.00273,
                 'initial_ETC_price': 5.73  ,
                  'initial_BTC_price': 8575.71,
                  'initial_LTC_price': 55,
                  'initial_BSV_price': 211,
                  'initial_ETH_price': 150,
                  'initial_BCH_price': 295
                }

#Create a dicitionary with a value corresponding to the qunatity of coins of each crypto
quantities = {'DOGE_quantity': my_crypto_currencies[6]['quantity'],
              'ETC_quantity': my_crypto_currencies[4]['quantity'],
              'BTC_quantity': my_crypto_currencies[3]['quantity'],
              'BCH_quantity': my_crypto_currencies[0]['quantity'],
              'LTC_quantity': my_crypto_currencies[1]['quantity'],
              'BSV_quantity': my_crypto_currencies[5]['quantity'],
              'ETH_quantity': my_crypto_currencies[2]['quantity']
             }

#Create a end point
end = time.time() + month #Using a month at a time due to this being a test

#Create a dictionary to keep the buying thresholds for each stock
crypto_buying_thresholds = {}
if (float(r.crypto.get_crypto_quote('LTC')['mark_price']) >= (buying_prices['initial_LTC_price'] * 1.0125)):
    r.orders.order_sell_crypto_by_quantity('LTC', my_crypto_currencies[1]['quantity'])
#While the current unix time is less than the current unix time plus a month of time
while time.time() < end:
    #So long as there are no errors
    try:
        try:
            try:
                #Create a dictionary to keep track of the asking price for each stock
                my_cryptos_asking_price = {}
                #Loop over all the names inside of the crypto_names list
                for name in range(len(crypto_names)):
                    #Get the price for the crypto in question and add it to the brypto_asking_price dictionary
                    ask_crypto_price = r.crypto.get_crypto_quote(crypto_names[name])['mark_price']
                    my_cryptos_asking_price[crypto_names[name]] = ask_crypto_price
                    sell_crypto = {'sell': float(ask_crypto_price) >= (buying_prices['initial_' + crypto_names[name] + '_price'] * 1.00175)}
                    #If the current asking price is more than or equal to the selling threshold
                    if (float(ask_crypto_price) >= (buying_prices['initial_' + crypto_names[name] + '_price'] * 1.05)):
                            print('sale of ' + crypto_names[name] + ' made!')
                            #Get the current crypto price
                            current_crypto_price = float(ask_crypto_price)
                            sale_price = {'sale_price': current_crypto_price}
                            #Sell my crypto coins
                            r.orders.order_sell_crypto_by_quantity(crypto_names[name],float(my_crypto_currencies[name]['quantity']) * 0.875, priceType='mark_price', timeInForce='gtc')
                            #print(((float(ask_crypto_price) >= (buying_prices['initial_' + crypto_names[name] + '_price'] * 1.0125)) or (float(ask_crypto_price) >= (buying_prices['initial_' + crypto_names[name] + '_price'] * .9))) and float(my_crypto_currencies[name]['quantity']) >= 0.00)
                            #Set a buy threshold for crypto coin
                            buy_threshold = current_crypto_price * .993
                            #Store buying threhshold in the crypto_buying_threshold dictionary
                            crypto_buying_thresholds[crypto_names[name]] = buy_threshold
                            #print('Made a sell of ' + crypto_names[name] + 'worth: ' + str(current_crypto_price - buying_prices['initial_' + crypto_names[name] + '_price']))
                            #Get the current price of crypto price
                            buying_prices['initial_' + crypto_names[name] + '_price'] = float(ask_crypto_price)

                    #If thre are any elements inside of the crypto_buying_thresholds dictionary and the name of the crypto is inside of the dictionary
                    if (len(crypto_buying_thresholds)) > 0 and (crypto_names[name] in crypto_buying_thresholds):
                            #If the current_crypto_price is less than the threshold set for buying crypto
                            print('Made buy for ' + crypto_names[name] + ' = ' + str((current_crypto_price <= crypto_buying_thresholds[crypto_names[name]]) and (float(account['buying_power'])> 0.06)))
                            if (current_crypto_price <= crypto_buying_thresholds[crypto_names[name]]) and (float(account['buying_power'])> 0.06):
                                #Buy the named crypto
                                r.orders.order_buy_crypto_by_price(crypto_names[name], float(account['buying_power']) * 0.99, priceType='mark_price', timeInForce='gtc')
                    #print(name + ': '  + ask_crypto_price)
            except TypeError:
                pass
        except ConnectionError:  
            pass
    #If there is an ValueError
    except ValueError:
        #Ignore error and move on
        pass
    print(my_cryptos_asking_price)

### CryptoTrader 1.0:

In [ ]:
##### Import the time library to be able to run the algorithm for a certain set of time.
import time

#Variables for common time data_types
hour = (60 * 60)
day = hour * 24
month = day * 30

#Create a end point
end = time.time() + month #Using a month at a time due to this being a test

#While the current unix time is less than the current unix time plus a month of time
while time.time() < end:
    #Try as long as no error pops up
    try:
        #Get prices for my cryptocurrency portfolio
        ask_DOGE_price = r.crypto.get_crypto_quote("DOGE")['ask_price']
        ask_BSV_price = r.crypto.get_crypto_quote("BSV")['ask_price']
        ask_ETH_price = r.crypto.get_crypto_quote("ETH")['ask_price']
        ask_BTC_price = r.crypto.get_crypto_quote("BTC")['ask_price']
        ask_ETC_price = r.crypto.get_crypto_quote("ETC")['ask_price']
        #print the prices of my cryptocurrency portfolio
        portfolio = {'DOGE': ask_DOGE_price,
                     'BSV': ask_BSV_price,
                     'ETH': ask_ETH_price,
                     'BTC': ask_BTC_price,
                     'ETC': ask_ETC_price}
        print(portfolio)
        #If the price of DOGE is less than or equal to 0.00318
        if float(ask_DOGE_price) >= 0.00318 and float(my_crypto_currencies[-1]['quantity']) >= 0:
            #Get the current price of DOGECOIN price
            current_DOGE_price = ask_DOGE_price
            #Sell my DOGE coins
            r.orders.order_sell_crypto_by_quantity('DOGE',my_crypto_currencies[-1]['quantity'], priceType='ask_price', timeInForce='gtc')
            #Set a buy threshold for DOGE coin
            buy_threshold = current_DOGE_price * 0.985
            #if the DOGECOIN asking price is less than or equal to the buying threshold
            if float(ask_DOGE_price) <= buy_threshold:
                #Buy 100 dollars woth of DOGECOIN
                r.orders.order_buy_crypto_by_price('DOGE', 80)
        #If the price of Bitcoin SV is more than or equal to 96.2
        if float(ask_BSV_price) >= 96.2 and float(my_crypto_currencies[-2]['quantity']) >= 0:
            #Get the current Bitcoin SV price
            current_BSV_price = ask_BSV_price
            #Sell my Bitcoin SV coins
            r.orders.order_sell_crypto_by_quantity('BSV',my_crypto_currencies[-2]['quantity'], priceType='ask_price', timeInForce='gtc')
            #Set the Bitcoin SV buying threshold
            BSV_buy_threshold = current_BSV_price * 0.985
            #if the Bitcoin SV asking price is less than or equal to the buying threshold for
            #Bitcoin SV
            if current_preicefloat(BSV_buy_threshold):
                #Buy 10 dollars worth of Bitcoin SV
                r.orders.order_buy_crypto_by_price('BSV', 8)
        #If the price of Etherium is less than or equal to 172.08 dollars
        print(float(ask_ETH_price) >= 172.08)
        print(float(my_crypto_currencies[1]['quantity']) >= 0)
        if float(ask_ETH_price) >= 172.08 and float(my_crypto_currencies[1]['quantity']):
            #Get the current Etherium price
            current_ETH_price = ask_ETH_price
            #Sell my Etherium coins
            r.orders.order_sell_crypto_by_quantity('ETH',my_crypto_currencies[1]['quantity'], priceType='ask_price', timeInForce='gtc')
        #If the price of Bitcoin is less than or equal to 8350 dollars
        if float(ask_BTC_price) >= 8350.00 and float(my_crypto_currencies[2]['quantity']) >= 0:
            #Get the current Bitcoin price
            current_BTC_price = ask_BTC_price
            #Sell my Bitcoin coins
            r.orders.order_sell_crypto_by_quantity('BTC',my_crypto_currencies[2]['quantity'], priceType='ask_price', timeInForce='gtc')
        #If the price of Etherium Classic is less than or equal to 4.53 dollars
        if float(ask_ETC_price) >= 4.53 and float(my_crypto_currencies[3]['quantity']) >= 0:
            #Get the current Etherium Classic price
            current_BSV_price = ask_BSV_price
            #Sell my Etherium Classic coins
            r.orders.order_sell_crypto_by_quantity('ETC',my_crypto_currencies[3]['quantity'], priceType='ask_price', timeInForce='gtc')
    #Except when a type error occurs
    except TypeError:
        #Simply move to the next api call
        pass